In [10]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import warnings


ModuleNotFoundError: No module named 'peewee'

In [ ]:

# Suppress specific warnings for cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', message="Liblinear failed to converge*") # Added for potential solver issues

# @title Data Download and Feature Engineering
# --- 1. Select ETF and Download Data ---
etf_ticker = "EWZ"
# Use dates roughly corresponding to the cleaned data in the paper
# Original paper data: 2009-12-12 to 2020-01-01 [cite: 43]
# Cleaned data starts later due to indicator calculation [cite: 89]
# Let's download a slightly wider range initially to allow for indicator calculation
start_date = "2009-01-01"
end_date = "2020-01-02" # End date is exclusive in yfinance usually

data = yf.download(etf_ticker, start=start_date, end=end_date)

print(f"Downloaded {len(data)} data points for {etf_ticker}")

# --- 2. Calculate Target Variable (Gamma) ---
# Target: 1 if Open(t) > Open(t-1), 0 otherwise
data['Gamma'] = (data['Open'].diff() > 0).astype(int)
# Shift target to align with previous day's features (predict t based on t-1 info)
data['Gamma'] = data['Gamma'].shift(-1)

# --- 3. Calculate Technical Indicators (Simplified Set) ---
# Using pandas_ta library, calculate a few example indicators
# This is a major simplification from the 216 used in the paper [cite: 77]
data.ta.sma(length=10, append=True) # Simple Moving Average
data.ta.rsi(length=14, append=True) # Relative Strength Index
data.ta.bbands(length=20, append=True) # Bollinger Bands (adds 3 columns)
data.ta.macd(append=True) # MACD (adds 3 columns)
data.ta.atr(append=True) # Average True Range

# Keep original OHLCV as features too
base_features = ['Open', 'High', 'Low', 'Close', 'Volume']
# Dynamically get added ta features (excluding potential NaNs like BB_RP)
ta_features = [col for col in data.columns if col.startswith(('SMA', 'RSI', 'BBP', 'BBB', 'BBM', 'MACD', 'MACDh', 'MACDs', 'ATRr'))]

all_feature_columns = base_features + ta_features

# --- Data Cleaning and Preparation ---
# Remove rows with NaN values created by indicators or target shift
data_clean = data.dropna()

# Separate features (X) and target (y)
X = data_clean[all_feature_columns].copy()
y = data_clean['Gamma'].copy()

# Scale features using Min-Max Scaler [cite: 83]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=all_feature_columns, index=X.index)

print(f"Data prepared with {len(X_scaled)} samples and {len(all_feature_columns)} features.")
print("Features:", all_feature_columns)


# @title Feature Selection (Pearson Correlation)
# --- 4. Feature Selection using Pearson's Correlation ---
# Calculate correlation between each feature and the target variable 'Gamma' [cite: 118]
correlations = X_scaled.corrwith(y).abs().sort_values(ascending=False)

print("\n--- Feature Correlation with Target (Absolute Values) ---")
print(correlations)

# Get the ranked list of features
ranked_features = correlations.index.tolist()


# @title Cross-Validation and Model Evaluation
# --- 5 & 6. Cross-Validation and MLP Training ---
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42) # Use shuffle for robustness

# Define feature subset sizes to test (e.g., top 5, 10, 15, all)
# Adjust based on the number of available features
max_features = len(ranked_features)
# Ensure subset sizes don't exceed max_features
subset_sizes = sorted(list(set([min(k, max_features) for k in [5, 10, 15, 20, 25, max_features]])))


results = {} # To store accuracy for each subset size

print(f"\n--- Running {n_splits}-Fold Cross-Validation ---")

for k in subset_sizes:
    selected_features = ranked_features[:k]
    print(f"Evaluating with Top {k} features: {selected_features}")

    X_subset = X_scaled[selected_features]
    fold_accuracies = []

    for fold, (train_index, test_index) in enumerate(skf.split(X_subset, y)):
        X_train, X_test = X_subset.iloc[train_index], X_subset.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Define MLP Classifier (parameters roughly based on paper's description)
        # Note: Hidden layer size calculation depends on number of features 'k' [cite: 106]
        # Paper used logistic activation, lbfgs solver, adaptive learning rate [cite: 95]
        # These params can be sensitive and may need tuning. LBFGS is better for smaller datasets.
        hidden_layer_size = int((k + len(np.unique(y))) / 2)
        mlp = MLPClassifier(hidden_layer_sizes=(max(1, hidden_layer_size),), # Ensure at least 1 neuron
                            activation='logistic',
                            solver='lbfgs', # Good for smaller datasets
                            max_iter=1000, # Increased iterations
                            learning_rate='adaptive',
                            learning_rate_init=0.03, # From paper [cite: 95]
                            momentum=0.9, # Default, paper used 0.2 [cite: 95] - keeping default for stability
                            random_state=fold, # Use fold number for reproducibility within CV
                            early_stopping=False # Paper ran tests with and without [cite: 175]
                           )

        # Train the model
        mlp.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = mlp.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        fold_accuracies.append(accuracy)

    # Calculate average accuracy for this feature subset size
    avg_accuracy = np.mean(fold_accuracies)
    results[k] = avg_accuracy
    print(f"Average Accuracy for Top {k} features: {avg_accuracy:.4f}")


# @title Results Table and Graph
# --- 7. Reproduce Table (Accuracy vs. Feature Count) ---
print("\n--- Results Summary (Accuracy vs. No. Top Features by Correlation) ---")
results_df = pd.DataFrame(list(results.items()), columns=['Num_Features', 'Avg_Accuracy'])
results_df['Avg_Accuracy'] = results_df['Avg_Accuracy'] * 100 # Display as percentage
print(results_df.to_string(index=False))

# --- 8. Reproduce Graph (Accuracy Gain Concept) ---
baseline_accuracy = results[max_features] # Accuracy using all calculated features
results_df['Accuracy_Gain (%)'] = results_df['Avg_Accuracy'] - (baseline_accuracy * 100)

plt.figure(figsize=(10, 6))
plt.plot(results_df['Num_Features'], results_df['Avg_Accuracy'], marker='o', linestyle='-')
plt.title(f'MLP Accuracy vs. Number of Top Features (EWZ, Selected by Correlation)')
plt.xlabel('Number of Top Features Used')
plt.ylabel('Average Cross-Validated Accuracy (%)')
plt.grid(True)
plt.xticks(results_df['Num_Features']) # Ensure ticks are at the tested feature counts
plt.show()

# Plot accuracy gain (similar concept to Figure 5 [cite: 180])
plt.figure(figsize=(10, 6))
plt.plot(results_df['Num_Features'], results_df['Accuracy_Gain (%)'], marker='o', linestyle='-', color='g')
plt.title(f'Accuracy Gain vs. Baseline (EWZ, Selected by Correlation)')
plt.xlabel('Number of Top Features Used')
plt.ylabel('Accuracy Gain vs. All Features (%)')
plt.grid(True)
plt.axhline(0, color='grey', linestyle='--', linewidth=0.8) # Add horizontal line at 0 gain
plt.xticks(results_df['Num_Features'])
plt.show()